In [2]:
from PIL import ImageChops
import math, operator
from PIL import Image
from functools import reduce
import imagehash

def rmsdiff(im1, im2):
    "Calculate the root-mean-square difference between two images"

    h = ImageChops.difference(im1, im2).histogram()

    # calculate rms
    return math.sqrt(reduce(operator.add,
        map(lambda h, i: h*(i**2), h, range(256))
    ) / (float(im1.size[0]) * im1.size[1]))

def rmsdiff2(img1, img2):
    return imagehash.average_hash(img1) - imagehash.average_hash(img2)

def equal(im1, im2):
    return ImageChops.difference(im1, im2).getbbox() is None

In [3]:
import sys

# from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average
import imageio


def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)

def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr
    
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

def compare_main(file1, file2):
    img1 = to_grayscale(imageio.imread(file1).astype(float))
    img2 = to_grayscale(imageio.imread(file2).astype(float))
    # compare
    return compare_images(img1, img2)

In [6]:
from os import listdir
from os.path import isfile, join
import timeit

path1 = '/home/dzanardo/github/apartamento/2449863607/'
path2 = '/home/dzanardo/github/apartamento/2454100925/'

files1 = [f for f in listdir(path1) if isfile(join(path1, f))]
files2 = [f for f in listdir(path2) if isfile(join(path2, f))]

In [4]:
file1 = "/home/dzanardo/github/apartamento/2449863607/14.jpg"
file2 = "/home/dzanardo/github/apartamento/2454100925/7.jpg"

im2 = Image.open(file1)
im1 = Image.open(file2)

print(rmsdiff(im2, im1))
print(rmsdiff2(im2, im1))

# read images as 2D arrays (convert to grayscale for simplicity)
n_m, n_0 = compare_main(file2, file1)
print("Manhattan norm:", n_m, "/ per pixel:", n_m/img1.size)
print("Zero norm:", n_0, "/ per pixel:", n_0*1.0/img1.size)

32.607018245906644
8


/home/dzanardo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: scipy.average is deprecated and will be removed in SciPy 2.0.0, use numpy.average instead
/home/dzanardo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  app.launch_new_instance()


NameError: name 'img1' is not defined

In [8]:
start = timeit.default_timer()
similares = 0

for file1 in files1:
    im2 = Image.open(path1 + file1)
    
    diff_p = 0
    print('--------------------------------- ' + str(file1))
    for file2 in files2:
        im1 = Image.open(path2 + file2)
        diff = rmsdiff(im2, im1)
        
        if diff < 40:
            similares += 1
            print(diff, file1, file2)
            break
            
print(similares)
stop = timeit.default_timer()
print(stop - start)


--------------------------------- 12.jpg
--------------------------------- 14.jpg
32.607018245906644 14.jpg 7.jpg
--------------------------------- 28.jpg
--------------------------------- 19.jpg
30.642151093079466 19.jpg 27.jpg
--------------------------------- 33.jpg
--------------------------------- 27.jpg
--------------------------------- 36.jpg
--------------------------------- 2.jpg
22.555963837300983 2.jpg 13.jpg
--------------------------------- 22.jpg
--------------------------------- 15.jpg
25.255473370839542 15.jpg 6.jpg
--------------------------------- 43.jpg
19.063775761802702 43.jpg 31.jpg
--------------------------------- 21.jpg
--------------------------------- 31.jpg
19.063775761802702 31.jpg 31.jpg
--------------------------------- 4.jpg
37.77758512001528 4.jpg 2.jpg
--------------------------------- 34.jpg
--------------------------------- 18.jpg
26.832498261678186 18.jpg 25.jpg
--------------------------------- 42.jpg
22.810953651293964 42.jpg 34.jpg
--------------

In [137]:
start = timeit.default_timer()
similares = 0

for file1 in files1:
    im2 = Image.open(path1 + file1)
    
    diff_p = 0
    #print('--------------------------------- ' + str(file1))
    for file2 in files2:
        im1 = Image.open(path2 + file2)
        diff = rmsdiff2(im2, im1)
        
        if diff < 5:
            similares += 1
            #print(diff, file1, file2)
            break
            

print(similares)
stop = timeit.default_timer()
print(stop - start)

23
3.514025796990609


In [135]:
start = timeit.default_timer()
similares = 0

for file1 in files1:
    im2 = path1 + file1
    
    diff_p = 0
    #print('--------------------------------- ' + str(file1))
    for file2 in files2:
        im1 = path2 + file2
        n_m, n_0 = compare_main(im1, im2)
        
        if n_m/img1.size < 30:
            similares += 1
            #print(n_m/img1.size, file1, file2)
            break
            

print(similares)
stop = timeit.default_timer()
print(stop - start)

28
8.612002982001286
